# Run a Documentation Template

This notebook shows how to run all tests defined in a documentation template.

## Before you begin

To use the ValidMind Developer Framework with a Jupyter notebook, you need to install and initialize the client library first, along with getting your Python environment ready.

If you don't already have one, you should also [create a documentation project](https://docs.validmind.ai/guide/create-your-first-documentation-project.html) on the ValidMind platform. You will use this project to upload your documentation and test results.

## Install the client library

In [18]:
%pip install --upgrade validmind

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Initialize the client library

In a browser, go to the **Client Integration** page of your documentation project and click **Copy to clipboard** next to the code snippet. This code snippet gives you the API key, API secret, and project identifier to link your notebook to your documentation project.

::: {.column-margin}
::: {.callout-tip}
This step requires a documentation project. [Learn how you can create one](https://docs.validmind.ai/guide/create-your-first-documentation-project.html).
:::
:::

Next, replace this placeholder with your own code snippet:

In [24]:
## Replace with code snippet from your documentation project ##

import validmind as vm

vm.init(
    api_host="https://api.prod.validmind.ai/api/v1/tracking",
    api_key="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
    api_secret="API_SECRET",
    project="xxxxxxxxxxxxxxxxxxxxxxxxx"
)

2023-07-10 13:57:08,779 - INFO - api_client - Connected to ValidMind. Project: Customer Churn Model - Test for Notebooks, Sydney (cljxccag7000016mkriy2l77l)


## Structure of a documentation template

All projects created in ValidMind are based on a documentation template. A documentation template is a collection of content blocks that, when rendered, produce a document that model developers can use for model validation.

The template structure is a simple combination of content `sections` where each section can have one or more content blocks. We currently support text-based content blocks (that are populated with the ValidMind UI) and test-driven content blocks. Under the hood, a template is represented as a YAML file. For more information about a template's structure, please refer to [this page](https://docs.validmind.ai/guide/edit-templates.html).

## Previewing a template

We can use the `preview_template()` function to preview the content blocks that will be populated by the developer framework.

In [20]:
vm.preview_template()

Accordion(children=(Accordion(children=(HTML(value='<p>Empty Section</p>'), Accordion(children=(HTML(value='<p…

## Running a template

We can use the `run_template()` function to run all tests defined in a documentation template. Note that each test in the template will have specific `context` requirements. Each context object can be passed as a function argument. For example, if a test requires a `model` object, we can pass it as a function argument:

```python
vm.run_template(model=vm_model) # vm_model is a validmind Model instance
```

Let's run through our customer churn demo to illustrate how to run a template and pass a model and dataset context to the `run_template()` function.

In [21]:
# Load a demo dataset and train a simple model
import xgboost as xgb
from validmind.datasets.classification import customer_churn as demo_dataset

df = demo_dataset.load_data()

vm_dataset = vm.init_dataset(
    dataset=df,
    target_column=demo_dataset.target_column,
    class_labels=demo_dataset.class_labels
)

train_df, validation_df, test_df = demo_dataset.preprocess(df)

x_train = train_df.drop(demo_dataset.target_column, axis=1)
y_train = train_df[demo_dataset.target_column]
x_val = validation_df.drop(demo_dataset.target_column, axis=1)
y_val = validation_df[demo_dataset.target_column]

model = xgb.XGBClassifier(early_stopping_rounds=10)
model.set_params(
    eval_metric=["error", "logloss", "auc"],
)
model.fit(
    x_train,
    y_train,
    eval_set=[(x_val, y_val)],
    verbose=False,
)

2023-07-10 13:56:10,183 - INFO - client - Pandas dataset detected. Initializing VM Dataset instance...
2023-07-10 13:56:10,188 - INFO - dataset - Inferring dataset types...


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=10,
              enable_categorical=False, eval_metric=['error', 'logloss', 'auc'],
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None, ...)

In [22]:
vm_train_ds = vm.init_dataset(
    dataset=train_df,
    type="generic",
    target_column=demo_dataset.target_column
)

vm_test_ds = vm.init_dataset(
    dataset=test_df,
    type="generic",
    target_column=demo_dataset.target_column
)

vm_model = vm.init_model(
    model,
    train_ds=vm_train_ds,
    test_ds=vm_test_ds,
)

2023-07-10 13:56:16,654 - INFO - client - Pandas dataset detected. Initializing VM Dataset instance...
2023-07-10 13:56:16,657 - INFO - dataset - Inferring dataset types...
2023-07-10 13:56:16,843 - INFO - client - Pandas dataset detected. Initializing VM Dataset instance...
2023-07-10 13:56:16,895 - INFO - dataset - Inferring dataset types...


Now we can run this project's documentation template by passing the required `dataset` and `model` context:

In [25]:
model_suite = vm.run_template(model=vm_model, dataset=vm_dataset)

KeyError: 'mean_of_values'